In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def get_crypto_data(crypto_id, days=365):
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': days,
        'interval': 'daily'
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    df = pd.DataFrame(data['prices'], columns=['timestamp', f'{crypto_id}_price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    
    df[f'{crypto_id}_volume'] = [x[1] for x in data['total_volumes']]
    df[f'{crypto_id}_market_cap'] = [x[1] for x in data['market_caps']]
    
    return df


In [3]:
def calculate_volatility(df, crypto_id,window=7):
    df[f'{crypto_id}_returns'] = df[f'{crypto_id}_price'].pct_change()
    df[f'{crypto_id}_volatility'] = df[f'{crypto_id}_returns'].rolling(window=window).std() * (365**0.5)
    return df

In [4]:
btc_data = get_crypto_data('bitcoin')

In [5]:
btc_data

,bitcoin_price,bitcoin_volume,bitcoin_market_cap
timestamp,,,
2023-10-11 00:00:00,27392.247703,1.015172e+10,5.352004e+11
2023-10-12 00:00:00,26842.190439,1.460844e+10,5.236564e+11
2023-10-13 00:00:00,26729.137206,1.086764e+10,5.216077e+11
2023-10-14 00:00:00,26841.136221,1.434441e+10,5.236556e+11
2023-10-15 00:00:00,26863.183569,5.543161e+09,5.239792e+11
...,...,...,...
2024-10-06 00:00:00,62091.932585,1.109545e+10,1.226762e+12
2024-10-07 00:00:00,62811.799728,1.459242e+10,1.241834e+12
2024-10-08 00:00:00,62287.390105,3.387888e+10,1.231092e+12


In [6]:
btc_data = calculate_volatility(btc_data,'bitcoin')

In [7]:
btc_data.head()

,bitcoin_price,bitcoin_volume,bitcoin_market_cap,bitcoin_returns,bitcoin_volatility
timestamp,,,,,
2023-10-11,27392.247703,1.015172e+10,5.352004e+11,NaN,NaN
2023-10-12,26842.190439,1.460844e+10,5.236564e+11,-0.020081,NaN
2023-10-13,26729.137206,1.086764e+10,5.216077e+11,-0.004212,NaN
2023-10-14,26841.136221,1.434441e+10,5.236556e+11,0.004190,NaN
2023-10-15,26863.183569,5.543161e+09,5.239792e+11,0.000821,NaN


In [8]:
btc_data.to_csv('crypto_market_data.csv')

In [9]:
btc_data.isnull().sum()

bitcoin_price         0
bitcoin_volume        0
bitcoin_market_cap    0
bitcoin_returns       1
bitcoin_volatility    7
dtype: int64

In [10]:
btc_data['bitcoin_returns']=btc_data['bitcoin_returns'].fillna(btc_data['bitcoin_returns'].mean())

In [11]:
btc_data['bitcoin_volatility']=btc_data['bitcoin_volatility'].fillna(btc_data['bitcoin_volatility'].mean())

In [12]:
btc_data

,bitcoin_price,bitcoin_volume,bitcoin_market_cap,bitcoin_returns,bitcoin_volatility
timestamp,,,,,
2023-10-11 00:00:00,27392.247703,1.015172e+10,5.352004e+11,0.002601,0.477486
2023-10-12 00:00:00,26842.190439,1.460844e+10,5.236564e+11,-0.020081,0.477486
2023-10-13 00:00:00,26729.137206,1.086764e+10,5.216077e+11,-0.004212,0.477486
2023-10-14 00:00:00,26841.136221,1.434441e+10,5.236556e+11,0.004190,0.477486
2023-10-15 00:00:00,26863.183569,5.543161e+09,5.239792e+11,0.000821,0.477486
...,...,...,...,...,...
2024-10-06 00:00:00,62091.932585,1.109545e+10,1.226762e+12,-0.000178,0.414264
2024-10-07 00:00:00,62811.799728,1.459242e+10,1.241834e+12,0.011594,0.438800
2024-10-08 00:00:00,62287.390105,3.387888e+10,1.231092e+12,-0.008349,0.358060


In [13]:
btc_data.to_csv('crypto_market_data.csv')